# CVML-Only Kill Test (Gemini Research, Round 2)
**Goal**: Test CVML (learnable feature mixing) WITHOUT CoRe on GRU.

**CoRe KILLED in Round 1**: Cosine similarity caused gradient singularity at zero predictions (loss spikes 17-22), t1 collapse (0.07-0.14 vs baseline 0.13-0.15), early stopping. All 3 AIs agree: drop CoRe entirely.

**Changes vs tightwd_v2 baseline**:
- CVML: 2-layer MLP residual block before input_proj (42->84->42, GELU, zero-init)
- No CoRe, no feature gate (pure isolation test)

**Kill test**: 3 seeds (s42/s43/s44), pass if mean val >= 0.2670 AND >= 2/3 seeds positive.
**Baseline**: tightwd_v2 ~0.266, pearson_v1 ~0.267

In [ ]:
# Cell 0: Mount Drive, download data from Kaggle
import os, json

from google.colab import drive
drive.mount('/content/drive')
os.makedirs('/content/drive/MyDrive/wunderfund', exist_ok=True)

!pip install -q kaggle==1.6.14 --force-reinstall
os.makedirs('/root/.kaggle', exist_ok=True)
with open('/root/.kaggle/kaggle.json', 'w') as f:
    json.dump({"username": "vincentvdo6", "key": "FILL_IN"}, f)
os.chmod('/root/.kaggle/kaggle.json', 0o600)

os.makedirs('/content/data', exist_ok=True)
!kaggle datasets download -d vincentvdo6/wunderfund-predictorium -p /content/data/ --force
!unzip -o -q /content/data/wunderfund-predictorium.zip -d /content/data/
!ls /content/data/*.parquet

In [ ]:
# Cell 1: Setup — clone repo, link data
import os, subprocess
REPO = "/content/competition_package"

os.chdir("/content")
os.system(f"rm -rf {REPO}")
os.system(f"git clone https://github.com/vincentvdo6/competition_package.git {REPO}")
os.chdir(REPO)
os.makedirs("datasets", exist_ok=True)
os.makedirs("logs", exist_ok=True)

os.system('ln -sf /content/data/train.parquet datasets/train.parquet')
os.system('ln -sf /content/data/valid.parquet datasets/valid.parquet')

assert os.path.exists("datasets/train.parquet"), "train.parquet not found!"
assert os.path.exists("datasets/valid.parquet"), "valid.parquet not found!"
print("Commit:", subprocess.check_output(["git", "rev-parse", "--short", "HEAD"], text=True).strip())
print(f"GPU: {os.popen('nvidia-smi --query-gpu=name --format=csv,noheader').read().strip()}")
print("Ready!")

In [ ]:
# Cell 2: Train 3 seeds with CVML-only config (no CoRe)
import os, subprocess, sys
os.chdir("/content/competition_package")

CONFIG = "configs/gru_cvml_only_v1.yaml"
SEEDS = [42, 43, 44]

print(f"CVML-Only Kill Test (no CoRe)")
print(f"Config: {CONFIG}")
print(f"Seeds: {SEEDS}")
print("=" * 60, flush=True)

for seed in SEEDS:
    print(f"\n{'='*60}")
    print(f"Training gru_cvml_only_v1 seed {seed}")
    print(f"{'='*60}", flush=True)
    proc = subprocess.Popen(
        [sys.executable, "-u", "scripts/train.py",
         "--config", CONFIG,
         "--seed", str(seed), "--device", "cuda"],
        stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True
    )
    for line in proc.stdout:
        print(line, end="", flush=True)
    proc.wait()
    if proc.returncode != 0:
        print(f"ERROR: seed {seed} failed with return code {proc.returncode}")

print(f"\nAll seeds done!")

In [ ]:
# Cell 3: Kill test evaluation
import os, glob, torch
os.chdir("/content/competition_package")

BASELINE_VAL = 0.2660  # tightwd_v2 average
PASS_THRESHOLD = 0.2670

results = []
for pt in sorted(glob.glob("logs/gru_cvml_only_v1_seed*.pt")):
    basename = os.path.basename(pt)
    if '_epoch' in basename:
        continue
    ckpt = torch.load(pt, map_location="cpu", weights_only=False)
    score = float(ckpt.get("best_score", 0))
    epoch = ckpt.get("best_epoch", "N/A")
    results.append((basename, score, epoch))

print(f"{'Model':<50} {'Val Score':>10} {'Epoch':>6} {'Delta':>8}")
print("-" * 80)
for name, score, epoch in results:
    delta = score - BASELINE_VAL
    tag = "  +" if delta > 0 else "  "
    print(f"{name:<50} {score:>10.4f} {str(epoch):>6} {tag}{delta:+.4f}")

scores = [s for _, s, _ in results]
mean_val = sum(scores) / len(scores) if scores else 0
positive = sum(1 for s in scores if s > BASELINE_VAL)

print(f"\n{'='*60}")
print(f"Mean val: {mean_val:.4f} (threshold: {PASS_THRESHOLD})")
print(f"Positive seeds: {positive}/{len(scores)} (need >=2/3)")
print(f"Delta vs baseline: {mean_val - BASELINE_VAL:+.4f}")

PASS = mean_val >= PASS_THRESHOLD and positive >= 2
print(f"\nKILL TEST: {'PASS' if PASS else 'FAIL'}")
if PASS:
    print("Next: Try CVML + SE-Net gate, then full seed expansion")
else:
    print("CVML KILLED. Next: try SE-Net gate only, or move to Week 2 plan")

In [ ]:
# Cell 4: Strip checkpoints + zip + save to Drive
import os, torch, glob, shutil
os.chdir("/content/competition_package")
os.makedirs("logs/slim", exist_ok=True)

for pt in sorted(glob.glob("logs/gru_cvml_only*.pt")):
    basename = os.path.basename(pt)
    if '_epoch' in basename:
        continue
    ckpt = torch.load(pt, map_location="cpu", weights_only=False)
    slim = {
        "model_state_dict": ckpt["model_state_dict"],
        "config": ckpt.get("config", {}),
        "best_score": ckpt.get("best_score", None),
        "best_epoch": ckpt.get("best_epoch", None),
    }
    out = f"logs/slim/{basename}"
    torch.save(slim, out)
    orig = os.path.getsize(pt) / 1e6
    new = os.path.getsize(out) / 1e6
    print(f"{basename}: {orig:.1f}MB -> {new:.1f}MB")

for npz in sorted(glob.glob("logs/normalizer_gru_cvml_only*.npz")):
    shutil.copy(npz, f"logs/slim/{os.path.basename(npz)}")
    print(f"Copied {os.path.basename(npz)}")

print(f"\n--- logs/slim/ contents ---")
for f in sorted(os.listdir("logs/slim")):
    sz = os.path.getsize(f"logs/slim/{f}") / 1e6
    print(f"  {f}: {sz:.1f}MB")

shutil.make_archive("/content/cvml_only_kill_test", "zip",
                     "/content/competition_package/logs/slim")
sz = os.path.getsize("/content/cvml_only_kill_test.zip") / 1e6
print(f"\ncvml_only_kill_test.zip: {sz:.1f}MB")

shutil.copy("/content/cvml_only_kill_test.zip",
            "/content/drive/MyDrive/wunderfund/cvml_only_kill_test.zip")
print("Saved to Drive: MyDrive/wunderfund/cvml_only_kill_test.zip")